In [1]:
import numpy as np
import pandas as pd

import lightgbm as lgb
import category_encoders as ce
from sklearn import metrics

In [2]:
data = pd.read_csv('kickstarter.csv', parse_dates=['launched'])
data.head(2)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00


In [3]:
data = data.query("state != 'live'")

In [4]:
data = data.assign(outcome=(data.state == 'successful').astype(int))
data = data.assign(hour=data.launched.dt.hour,
                  day=data.launched.dt.day,
                  month=data.launched.dt.month,
                  year=data.launched.dt.year)
data.head(2)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0,12,11,8,2015
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0,4,2,9,2017


In [5]:
num_X = data[['hour', 'day', 'month', 'year', 'goal', 'outcome']]
num_X.head(3)

,hour,day,month,year,goal,outcome
0,12,11,8,2015,1000.0,0
1,4,2,9,2017,30000.0,0
2,0,12,1,2013,45000.0,0


In [6]:
cat_features = ['category', 'currency', 'country']

In [7]:
X = num_X.join(data[cat_features])
X.head(2)

,hour,day,month,year,goal,outcome,category,currency,country
0,12,11,8,2015,1000.0,0,Poetry,GBP,GB
1,4,2,9,2017,30000.0,0,Narrative Film,USD,US


In [8]:
valid_fraction = 0.1
valid_size = int(len(X)*valid_fraction)

train = X[: -2 * valid_size]
valid = X[-2 * valid_size : -valid_size]
test = X[-valid_size:]
train.shape

(300690, 9)

In [9]:
encoder = ce.CatBoostEncoder(cols=cat_features)

In [10]:
encoder.fit(train[cat_features], train['outcome'])

CatBoostEncoder(a=1, cols=['category', 'currency', 'country'],
                drop_invariant=False, handle_missing='value',
                handle_unknown='value', random_state=None, return_df=True,
                sigma=None, verbose=0)

In [11]:
encoded_tr = train.join(encoder.transform(train[cat_features]).add_suffix('_catboost'))
encoded_va = valid.join(encoder.transform(valid[cat_features]).add_suffix('_catboost'))
encoded_te = test.join(encoder.transform(test[cat_features]).add_suffix('_catboost'))

encoded_tr.head(2)

,hour,day,month,year,goal,outcome,category,currency,country,category_catboost,currency_catboost,country_catboost
0,12,11,8,2015,1000.0,0,Poetry,GBP,GB,0.360187,0.357122,0.361636
1,4,2,9,2017,30000.0,0,Narrative Film,USD,US,0.384609,0.373392,0.376631


In [12]:
features = encoded_tr.columns.drop(['outcome', 'category', 'currency', 'country'])
features

Index(['hour', 'day', 'month', 'year', 'goal', 'category_catboost',
       'currency_catboost', 'country_catboost'],
      dtype='object')

In [13]:
param = {'num_leaves': 65, 'objective': 'binary', 'metric': 'auc'}
num_rounds = 1000

In [14]:
dtrain = lgb.Dataset(encoded_tr[features], encoded_tr['outcome'])
dvalid = lgb.Dataset(encoded_va[features], encoded_va['outcome'])

In [15]:
model = lgb.train(param, dtrain, num_rounds, valid_sets=[dvalid], early_stopping_rounds=5)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027761 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 531
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
[1]	valid_0's auc: 0.723307
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.724934
[3]	valid_0's auc: 0.726201
[4]	valid_0's auc: 0.727114
[5]	valid_0's auc: 0.729399
[6]	valid_0's auc: 0.730438
[7]	valid_0's auc: 0.73091
[8]	valid_0's auc: 0.731443
[9]	valid_0's auc: 0.731882
[10]	valid_0's auc: 0.732392
[11]	valid_0's auc: 0.732735
[12]	valid_0's auc: 0.733353
[13]	valid_0's auc: 0.733801
[14]	valid_0's auc: 0.733984
[15]	valid_0's auc: 0.734442
[16]	valid_0's au

In [16]:
preds = model.predict(encoded_te[features])
score = metrics.roc_auc_score(encoded_te['outcome'], preds)
score 

0.7460185532865766